In [1]:
#from __future__ import division
import sys
!{sys.executable} -m pip install numpy

The directory '/home/varshini/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/varshini/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.


In [2]:

import numpy
from python_speech_features import sigproc
from scipy.fftpack import dct

ModuleNotFoundError: No module named 'python_speech_features'

In [3]:
!{sys.executable} -m pip install python_speech_features

The directory '/home/varshini/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/varshini/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
  Running setup.py install for python-speech-features ... done


In [4]:
!{sys.executable} -m pip install scipy.fftpack

The directory '/home/varshini/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/varshini/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
  Could not find a version that satisfies the requirement scipy.fftpack (from versions: )
No matching distribution found for scipy.fftpack


In [5]:
from python_speech_features import sigproc
from scipy.fftpack import dct

ModuleNotFoundError: No module named 'scipy'

In [6]:
!{sys.executable} -m pip install scipy

The directory '/home/varshini/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/varshini/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
    100% |████████████████████████████████| 31.2MB 32kB/s eta 0:00:011  23% |███████▍                        | 7.2MB 7.1MB/s eta 0:00:04    36% |███████████▉                    | 11.5MB 4.7MB/s eta 0:00:05


In [7]:
from python_speech_features import sigproc
from scipy.fftpack import dct

In [34]:
def mfcc(signal, samplerate=16000, winlen=0.025, winstep=0.01, numcep=13, nfilt=26, nfft=512, lowfreq=0, highfreq=None,preemph=0.97,ceplifter=22, appendEnergy=True, winfunc=lambda x:numpy.ones((x,))):
    feat, energy = fbank(signal,samplerate,winlen,winstep,nfilt,nfft,lowfreq,highfreq,preemph,winfunc)
    feat = numpy.log(feat)
    feat = dct(feat, type=2, axis=1, norm='ortho')[:,:numcep]#To store only the first 13 filter coefficients
    feat=lifter(feat,ceplifter)
    if appendEnergy:
        feat[:,0] = numpy.log(energy) #Replacing first cepstral coefficient with log of frame energy
    return feat

In [9]:
def fbank(signal, samplerate=16000, winlen=0.025, winstep=0.01, nfilt=26, nfft=512, lowfreq=0, highfreq=None, preemph=0.97, winfunc=lambda x:numpy.ones((x,))):
    highfreq = highfreq or samplerate/2
    signal = sigproc.preemphasis(signal, preemph)
    frames = sigproc.framesig(signal, winlen*samplerate, winstep*samplerate, winfunc)
    pspec = sigproc.powspec(frames, nfft)
    energy = numpy.sum(pspec,1)#Stores energy in each frame
    energy = numpy.where(energy==0,numpy.finfo(float).eps,energy)
    
    fb = get_filterbanks(nfilt, nfft, samplerate, lowfreq, highfreq)
    feat = numpy.dot(pspec, fb.T) #Compute filterbank energies
    feat = numpy.where(feat==0, numpy.finfo(float).eps, feat)
    
    return feat, energy

In [10]:
def logfbank(signal, samplerate=16000, winlen=0.025, winstep=0.01, nfilt=26,nfft=512, lowfreq=0, highfreq=None, preemph=0.97, winfunc=lambda x:numpy.ones((x,))):
    feat, energy = fbank(signal, samplerate, winlen, winstep, nfilt, nfft, lowfreq, highfreq, preemph, winfunc)
    return numpy.log(feat)

In [15]:
def ssc(signal, samplerate=16000, winlen=0.025, winstep=0.01,nfilt=26,nfft=512,lowfreq=0,highfreq=None,preemph=0.97,winfunc=lambda x:numpy.ones((x,))):
    signal = sigproc.preemphasis(signal,preemph)
    frames = sigproc.framesig(signal, winlen*samplerate, winstep*samplerate, winfunc)
    pspec = sigproc.powspec(frames,nfft)
    pspec = numpy.where(pspec == 0,numpy.finfo(float).eps,pspec) # if things are all zeros we get problems

    fb = get_filterbanks(nfilt,nfft,samplerate,lowfreq,highfreq)
    feat = numpy.dot(pspec,fb.T) # compute the filterbank energies
    R = numpy.tile(numpy.linspace(1,samplerate/2,numpy.size(pspec,1)),(numpy.size(pspec,0),1))

    return numpy.dot(pspec*R,fb.T) / feat

In [16]:
def hz2mel(hz):
    return 2595*numpy.log10(1+hz/700.)

In [17]:
def mel2hz(mel):
    return 700*(10**(mel/2595.0)-1)

In [18]:
def get_filterbanks(nfilt=20, nfft=512, samplerate=16000, lowfreq=0, highfreq=None):
    highfreq=highfreq or samplerate/2
    assert highfreq<=samplerate/2, "highfreq is greater than samplerate/2"
    
    lowmel=hz2mel(lowfreq)
    highmel=hz2mel(highfreq)
    melpoints=numpy.linspace(lowmel, highmel, nfilt+2)
    #Convert from Hz to FFT bin number
    bin = numpy.floor((nfft+1)*mel2hz(melpoints)/samplerate)
    
    fbank = numpy.zeros([nfilt,nfft//2+1])
    for j in range(0,nfilt):
        for i in range(int(bin[j]),int(bin[j+1])):
            fbank[j,i] = (i-bin[j])/(bin[j+1]-bin[j])
        for i in range(int(bin[j+1]), int(bin[j+2])):
            fbank[j,i] = (bin[j+2]-i)/(bin[j+2]-bin[j+1])
    return fbank

In [19]:
def lifter(cepstra, L=22):
    if L>0:
        nframes,ncoeff = numpy.shape(cepstra)
        n = numpy.arange(ncoeff)
        lift = 1+(L/2.)*numpy.sin(numpy.pi*n/L)
        return lift*cepstra
    
    else:
        return cepstra

In [20]:
def delta(feat, N):
    if N<1:
        raise ValueError('N must be an integer>=1')
    NUMFRAMES = len(feat)
    denominator = 2*sum([i**2 for i in range(1,N+1)])
    delta_feat = numpy.empty_like(feat)
    padded = numpy.pad(feat,((N,N),(0,0)),mode='edge')
    for t in range(NUMFRAMES):
        delta_feat[t] = numpy.dot(numpy.arange(-N,N+1), padded[t:t+2*N+1])/denominator
    return delta_feat
        

In [21]:
import decimal
import math
import logging

In [22]:
def round_half_up(number):
    return int(decimal.Decimal(number).quantize(decimal.Decimal('1'),rounding=decimal.ROUND_HALF_UP))


In [23]:
def rolling_window(a, window, step=1):
    shape = a.shape[:-1]+(a.shape[-1]-window+1, window)
    strides = a.strides+(a.strides[-1],)
    return numpy.lib.stride_tricks.as_strided(a,shape=shape,strides=strides)[::step]

In [25]:
def framesig(sig, frame_len, frame_step, winfunc=lambda x:numpy.ones((x,)),stride_trick=True):
    slen=len(sig)
    frame_len=int(round_half_up(frame_len))
    frame_step=int(round_half_up(frame_step))
    if slen<=frame_len:
        numframes=1
    else:
        numframes=1+int(math.ceil((1.0*slen-frame_len)/frame_step))
        
    padlen = int((numframes-1)*frame_step+frame_len)
    
    zeros=numpy.zeros((padlen-slen,))
    padsignal=numpy.concatenate((sig,zeros))
    if stride_trick:
        win=winfunc(frame_len)
        frames=rolling_window(padsignal,window=frame_len,step=frame_step)
    else:
        indices = numpy.tile(numpy.arange(0, frame_len), (numframes, 1)) + numpy.tile(numpy.arange(0, numframes * frame_step, frame_step), (frame_len, 1)).T
        indices = numpy.array(indices, dtype=numpy.int32)
        frames = padsignal[indices]
        win = numpy.tile(winfunc(frame_len), (numframes, 1))
    return frames*win

In [26]:
def deframesig(frames, siglen, frame_len, frame_step, winfunc=lambda x:numpy.ones((x,))):
    frame_len = round_half_up(frame_len)
    frame_step = round_half_up(frame_step)
    numframes = numpy.shape(frames)[0]
    assert numpy.shape(frames)[1] == frame_len, '"frames" matrix is wrong size, 2nd dim is not equal to frame_len'

    indices = numpy.tile(numpy.arange(0, frame_len), (numframes, 1)) + numpy.tile(
        numpy.arange(0, numframes * frame_step, frame_step), (frame_len, 1)).T
    indices = numpy.array(indices, dtype=numpy.int32)
    padlen = (numframes - 1) * frame_step + frame_len

    if siglen <= 0: siglen = padlen

    rec_signal = numpy.zeros((padlen,))
    window_correction = numpy.zeros((padlen,))
    win = winfunc(frame_len)

    for i in range(0, numframes):
        window_correction[indices[i, :]] = window_correction[
                                               indices[i, :]] + win + 1e-15  # add a little bit so it is never zero
        rec_signal[indices[i, :]] = rec_signal[indices[i, :]] + frames[i, :]

    rec_signal = rec_signal / window_correction
    return rec_signal[0:siglen]

In [27]:
def magspec(frames, NFFT):
    if numpy.shape(frames)[1]>NFFT:
        logging.warn('frame length (%d) is greater than FFT size (%d), frame will be truncated. Increase NFFT to avoid.',numpy.shape(frames)[1], NFFT)
        complex_spec = numpy.fft.rfft(frames, NFFT)
        return numpy.absolute(complex_spec)

In [28]:
def powspec(frames, NFFT):
    return 1.0/NFFT*numpy.square(magspec(frames, NFFT))

In [30]:
def logpowspec(frames,NFFT,norm=1):
    ps=powspec(frames, NFFT)
    ps[ps<=1e-30]=1e-30
    lps=10*numpy.log10(ps)
    if norm:
        return lps-numpy.max(lps)
    else:
        return lps

In [31]:
def preemphasis(signal,coeff=0.95):
    return numpy.append(signal[0], signal[1:]-coeff*signal[:-1])

In [32]:
import scipy.io.wavfile as wav

In [35]:
(rate, sig) = wav.read("female.wav")
mfcc_feat=mfcc(sig,rate)
d_mfcc_feat=delta(mfcc_feat,2)
fbank_feat=logfbank(sig,rate)

print(fbank_feat[1:3,:])

[[ 7.31980869  7.78864759  8.42487588  8.49527852  8.63043262  8.87487021
   9.6229585   9.41699154  9.83313432 10.3540489  11.10170434 11.71867703
  11.41783178 11.89420804 11.65078413 13.34874206 13.87114465 13.68725636
  12.35856915 12.20042134 12.78460616 12.41508887 11.98950662 11.59510427
  11.39142357 11.13869419]
 [ 9.05419707  9.48662175 10.13588777 10.90790232 11.18087465 10.57921129
  10.4586     11.19639412 11.73661073 11.84598679 11.57982299 12.10852458
  12.00034944 12.43780441 12.2627719  13.58807688 13.94485583 13.98827067
  12.95248628 12.48481096 13.53469224 12.91089447 11.69212739 11.23966473
  11.01000797 10.88051898]]
